Machine Learning Pemula - Sinatrio Bimo Wahyudi

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

Extract Datasets File

In [ ]:
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Data Preparation

In [ ]:
base_dir = '/tmp/rockpaperscissors'
train_dir = os.path.join(base_dir, 'train') 
val_dir = os.path.join(base_dir, 'val')

In [ ]:
os.mkdir(train_dir)
os.mkdir(val_dir)

In [ ]:
paper_directory = os.path.join(base_dir, 'paper')
rock_directory = os.path.join(base_dir, 'rock')
scissors_directory = os.path.join(base_dir, 'scissors')

In [ ]:
paper_train = os.path.join(train_dir, 'paper')
rock_train = os.path.join(train_dir, 'rock')
scissors_train = os.path.join(train_dir, 'scissors')

In [ ]:
paper_val = os.path.join(val_dir, 'paper')
rock_val = os.path.join(val_dir, 'rock')
scissors_val = os.path.join(val_dir, 'scissors')

In [ ]:
os.mkdir(paper_train)
os.mkdir(rock_train)
os.mkdir(scissors_train)

os.mkdir(paper_val)
os.mkdir(rock_val)
os.mkdir(scissors_val)

In [ ]:
paper_trainf, paper_valf = train_test_split(os.listdir(paper_directory), test_size=0.4)
rock_trainf, rock_valf = train_test_split(os.listdir(rock_directory), test_size=0.4)
scissors_trainf, scissors_valf = train_test_split(os.listdir(scissors_directory), test_size=0.4)

In [ ]:
import shutil
for filename in paper_trainf:
  shutil.copy(os.path.join(paper_directory, filename), os.path.join(paper_train, filename))
for filename in rock_trainf:
  shutil.copy(os.path.join(rock_directory, filename), os.path.join(rock_train, filename))
for filename in scissors_trainf:
  shutil.copy(os.path.join(scissors_directory, filename), os.path.join(scissors_train, filename))
for filename in paper_valf:
  shutil.copy(os.path.join(paper_directory, filename), os.path.join(paper_val, filename))
for filename in rock_valf:
  shutil.copy(os.path.join(rock_directory, filename), os.path.join(rock_val, filename))
for filename in scissors_valf:
  shutil.copy(os.path.join(scissors_directory, filename), os.path.join(scissors_val, filename))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

CNN Architecture

In [ ]:
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(32, (3,3), activation='relu',input_shape=(150,150,3)),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Dropout(0.3),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(),
    metrics=['accuracy']
)

Training Model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch = 25,
    epochs = 20,
    validation_data = val_generator,
    validation_steps=5,
    verbose=2)

Uploader File

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  print(fn)

  if classes[0][0] == 1.0:
    print('paper')
  elif classes[0][1] == 1.0:
    print('rock')
  elif classes[0][2] == 1.0:
    print('scissors')